<a href="https://colab.research.google.com/github/poojashreeNS/ANN_Techniques/blob/main/ANN_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install faiss-cpu

In [ ]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
import pickle

In [ ]:
stackexchange = fetch_movielens()
train = stackexchange['train']
test = stackexchange['test']

model = LightFM(learning_rate=0.05, loss='warp', no_components=64, item_alpha=0.001)
model.fit_partial(train, item_features=stackexchange['item_features'], epochs=20 )

item_vectors = stackexchange['item_features'] * model.item_embeddings

In [ ]:
with open('stackexchange.pickle', 'wb') as f:
    pickle.dump({"name": stackexchange['item_feature_labels'], "vector": item_vectors}, f)

In [ ]:
def load_data():
    with open('stackexchange.pickle', 'rb') as f:
        data = pickle.load(f)
    return data

data = load_data()
data

{'name': array(['Toy Story (1995)', 'GoldenEye (1995)', 'Four Rooms (1995)', ...,
        'Sliding Doors (1998)', 'You So Crazy (1994)',
        'Scream of Stone (Schrei aus Stein) (1991)'], dtype=object),
 'vector': array([[-0.31464535,  0.13365287,  0.38005927, ...,  0.02249122,
         -0.03750825, -0.08592322],
        [-0.16997173,  0.11235693,  0.2217186 , ..., -0.13605641,
          0.2161419 ,  0.11136161],
        [-0.12343781,  0.14566147,  0.05448459, ...,  0.41542345,
          0.0528531 , -0.07463649],
        ...,
        [ 0.07096124, -0.18025255, -0.05786032, ..., -0.01613087,
          0.04188032,  0.08435941],
        [-0.00207763, -0.06229061, -0.04819448, ..., -0.18208297,
         -0.00321682,  0.21578231],
        [ 0.05797838, -0.05754833, -0.0014502 , ..., -0.0947851 ,
          0.03941731,  0.10738818]], dtype=float32)}

# **LSH**

LSH construct a hash table as their data structure by mapping points that are nearby into the same bucket.

In LSH, in order to construct the index, we apply multiple hash functions to map data points into buckets so that data points near each other are located in the same buckets with high probability, while data points far from each other are likely to fall into different buckets.


In order to search the constructed index, the query point is hashed in order to obtain the closest buckets (a set of candidate points) from which the closest to the query points are returned.

In [ ]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimension, num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = LSHIndex(data["vector"], data["name"])
index.build()

In [ ]:
movie_index = 90
movie_vector = data['vector'][movie_index:movie_index+1]
print(f"The most simillar movies to {data['name'][movie_index]} are:")
index.query(movie_vector)

The most simillar movies to Nightmare Before Christmas, The (1993) are:


['Shawshank Redemption, The (1994)',
 'Babe (1995)',
 'Lion King, The (1994)',
 'Apollo 13 (1995)',
 'Braveheart (1995)',
 'Fugitive, The (1993)',
 'Nightmare Before Christmas, The (1993)',
 'Terminator 2: Judgment Day (1991)',
 'Pulp Fiction (1994)',
 'Star Wars (1977)']

# **Exhaustive Search**

Comparing each point to every other point, which will require Linear query time depending on the size of dataset.

In [ ]:
class BruteForceIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        # I expect only query on one vector thus the slice
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = BruteForceIndex(data["vector"], data["name"])
index.build()

In [ ]:
movie_index = 90
movie_vector = data['vector'][movie_index:movie_index+1]
print(f"The most simillar movies to {data['name'][movie_index]} are:")
index.query(movie_vector)

The most simillar movies to Nightmare Before Christmas, The (1993) are:


['Nightmare Before Christmas, The (1993)',
 'Heavy Metal (1981)',
 'Benny & Joon (1993)',
 'Heathers (1989)',
 'Pink Floyd - The Wall (1982)',
 'Akira (1988)',
 'Fantasia (1940)',
 'Winnie the Pooh and the Blustery Day (1968)',
 'Sneakers (1992)',
 'Hudsucker Proxy, The (1994)']

# **Product Quantization**





In Product Quantization we can increase drastically the number of centroids by dividing each vector into many vectors and run our quantizer on all of these and thus improves accuracy.

Construct a table with the calculated distance between each sub-vector and each of the centroids for that sub-vector.

Calculating approximate distance values for each of the vectors in the dataset, we just use those centroids id’s to look up the partial distances in the table and sum those up!

In [ ]:
class IVPQIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        quantizer = faiss.IndexFlatL2(self.dimention)
        self.index = faiss.IndexIVFPQ(quantizer, 
                                      self.dimention, 
                                      number_of_partition, 
                                      search_in_x_partitions, 
                                      subvector_size)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [ ]:
index = IVPQIndex(data["vector"], data["name"])
index.build()

In [ ]:
movie_index = 90
movie_vector = data['vector'][movie_index:movie_index+1]
print(f"The most simillar movies to {data['name'][movie_index]} are:")
index.query(movie_vector)

The most simillar movies to Nightmare Before Christmas, The (1993) are:


['Nightmare Before Christmas, The (1993)',
 'Heavy Metal (1981)',
 'Fantasia (1940)',
 'Pink Floyd - The Wall (1982)',
 'Batman (1989)',
 'E.T. the Extra-Terrestrial (1982)',
 'Return of the Jedi (1983)',
 'Aladdin (1992)',
 'Heathers (1989)',
 'Star Wars (1977)']

# **Trees based ANN**

They construct forests (collection of trees) as their data structure by splitting the dataset into subsets.

Each tree is constructed in the following way, we pick two points at random and split the space into two by their hyperplane, we keep splitting into the subspaces recursively until the points associated with a node is small enough.

In order to search the constructed index, the forest is traversed in order to obtain a set of candidate points from which the closest to the query point is returned.

In [ ]:
import annoy

class AnnoyIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention)
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [ ]:
index = AnnoyIndex(data["vector"], data["name"])
index.build()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
movie_vector, movie_name = data['vector'][90], data['name'][90]
simlar_movies_names = '\n* '.join(index.query(movie_vector))
print(f"The most similar movies to {movie_name} are:\n* {simlar_movies_names}")

The most similar movies to Nightmare Before Christmas, The (1993) are:
* Nightmare Before Christmas, The (1993)
* Heavy Metal (1981)
* Fantasia (1940)
* Aladdin (1992)
* Benny & Joon (1993)
* Beauty and the Beast (1991)
* Lion King, The (1994)
* Empire Strikes Back, The (1980)
* Jurassic Park (1993)
* Sound of Music, The (1965)


# **HNSW**

The algorithm examines the distances from a query to the neighbors of a current base node and then selects as the next base node the adjacent node that minimizes the distance, while constantly keeping track of the best-discovered neighbors. The search is terminated when some stopping condition is met.

In [ ]:
!pip install nmslib
import nmslib

class NMSLIBIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels
    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

     |████████████████████████████████| 13.5 MB 70 kB/s 
     |████████████████████████████████| 188 kB 47.9 MB/s 


In [ ]:
index = NMSLIBIndex(data["vector"], data["name"])
index.build()

In [ ]:
movie_index = 90
movie_vector = data['vector'][movie_index:movie_index+1]
print(f"The most simillar movies to {data['name'][movie_index]} are:")
index.query(movie_vector)

The most simillar movies to Nightmare Before Christmas, The (1993) are:


['Nightmare Before Christmas, The (1993)',
 'Heavy Metal (1981)',
 'Fantasia (1940)',
 'Pink Floyd - The Wall (1982)',
 'Batman (1989)',
 'E.T. the Extra-Terrestrial (1982)',
 'Return of the Jedi (1983)',
 'Aladdin (1992)',
 'Heathers (1989)',
 'Star Wars (1977)']